In [ ]:
import sys
import os
import re
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
import plotly.express as px
import numpy as np

from scripts.accent_cleaner import AccentCleaner
from scripts.column_aligner import ColumnAligner
from scripts.utils import split_at_char, replace_with

In [ ]:
df_raw = pd.read_csv("../data/raw/68542.csv", sep=";", decimal=",")

In [ ]:
df_raw.head()

In [ ]:
df_raw.info()

In [ ]:
df_raw.sample(15)

In [ ]:
df_raw.drop(columns=['National Total'], inplace=True)

In [ ]:
df_raw.isnull().sum()

In [ ]:
df_municipalities_null = df_raw[df_raw['Municipalities'].isnull()]
df_municipalities_null[df_municipalities_null["Provinces"] == "08 Barcelona"]

### We have information about provinces without the municipalities, we should drop them

In [ ]:
df_raw[df_raw['Provinces'] == "25 Lleida"].sample(5)

In [ ]:

prov_null = df_raw[["Provinces", "Municipalities"]]

prov_null[(prov_null["Provinces"].isnull()) & (prov_null["Municipalities"].isnull())]

In [ ]:

prov_null[(prov_null["Provinces"].isnull())]

In [ ]:
df_raw[df_raw['Total'].isnull()].sample(30)

In [ ]:
df_raw = df_raw.dropna(subset=["Provinces", "Municipalities", "Total"], how="any")

In [ ]:
df_2024 = df_raw[df_raw['Periodo'] == 2024].copy()

In [ ]:
df_2024.reset_index(drop=True, inplace=True)

In [ ]:
idx = pd.Index(["Provinces", "Municipalities", "Sex", "Age", "Periodo", "Total"])
eng = pd.Index(["province", "municipality", "sex", "age", "year", "total"])

df_2024.rename(columns=dict(zip(idx, eng)), inplace=True)

### Get the correct types

In [ ]:
df_2024["total"] = pd.to_numeric(df_2024["total"].str.replace(".", "", regex=False)).astype("int32")

In [ ]:
df_2024 = df_2024[df_2024["age"] != "All ages"]

In [ ]:
(df_2024["age"].str.contains(r"^\d"))

In [ ]:
from scripts.utils import split_column_at

df_2024["cprov"] = split_column_at(df_2024, "province", " ", index=0)
df_2024["province"] = split_column_at(df_2024, "province", " ", index=1)
df_2024["cmun"] = split_column_at(df_2024, "municipality", " ", index=0)
df_2024["municipality"] = split_column_at(df_2024, "municipality", " ", index=1)
df_2024["age"] = split_column_at(df_2024, "age", " ", index=0)

In [ ]:
df_2024["age"] = pd.to_numeric(df_2024["age"].str.replace(".", "", regex=False)).astype("int32")

In [ ]:
df_2024["cprov"] = pd.to_numeric(df_2024["cprov"].str.replace(".", "", regex=False)).astype("int32")
df_2024["cmun"] = pd.to_numeric(df_2024["cmun"].str.replace(".", "", regex=False)).astype("int32")

In [ ]:
cleaner = AccentCleaner([df_2024], ['municipality', 'province'])
cleaner.cleanAccents()